In [1]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
def create_text(theme):
    quality =0
    text = generate(theme)
    review = review_text(text)

In [2]:
# 1つ目のチェーン : 劇のタイトルと時代からあらすじを生成

# LLMの準備
llm = OpenAI(temperature=.7)

# テンプレートの準備
# テンプレートの準備
template = """あなたは劇作家です。劇のタイトルと時代が与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
時代:{era}
あらすじ:"""

# プロンプトテンプレートの生成
prompt_template = PromptTemplate(
    input_variables=["title", 'era'], 
    template=template
)

# LLMChainの準備
synopsis_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    output_key="synopsis"
)

In [3]:
# 2つ目のチェーン : 劇のあらすじからレビューを生成

# LLMの準備準備
llm = OpenAI(temperature=.7)

# テンプレートの準備
template = """あなたは演劇評論家です。 劇のあらすじが与えられた場合、そのあらすじのレビューを書くのがあなたの仕事です。

あらすじ:
{synopsis}
レビュー:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
review_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="review"
)

In [4]:
from langchain.chains import SequentialChain

# SequentialChainで2つのチェーンを繋ぐ
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    output_variables=["synopsis", "review"],
    verbose=True
)

In [5]:
# SequentialChainの実行
review = overall_chain({"title":"浜辺の夕暮れ時の悲劇", "era": "戦国時代"})



> Entering new SequentialChain chain...

> Finished chain.


In [6]:
# レビューの表示
print(review)

{'title': '浜辺の夕暮れ時の悲劇', 'era': '戦国時代', 'synopsis': '\n\n戦国時代、ある小さな町に住む一家の若い夫婦が悲劇に遭う。一家は毎日を楽しむために、江戸から馬車を手に入れて町を観光しに行った。だが、若い夫婦を待つのは敵軍の攻撃だった。敵軍は町を焼き払い、若い夫婦を捕まえることに成功した。夫婦は仲間たちと一緒に洞窟に逃げ込んだが、敵軍は洞窟に火をつけてしまい、若い夫婦を殺してしまった。', 'review': '\n\n「戦国時代の小さな町」は、非常に深刻な悲劇を描いた本作品です。若い夫婦が、安全な観光を楽しむために江戸から馬車を手に入れた時、突然の敵軍の攻撃に見舞われてしまうのです。敵軍は町を焼き、若い夫婦を捕まえることに成功します。しかし、彼らは仲間と一緒に洞窟に逃げ込んだものの、敵軍は洞窟に火をつけてしまい、若い夫婦を殺してしまいま'}
